In [1]:
# Sistemas de Recomendação
# Filtragem Colaborativa (User-Based) com surprise
# Objetivo: Recomendar produtos com base na similaridade entre usuários e em avaliações anteriores.
# https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset
# O arquivo u.data tem colunas separadas por tab (\t): user_id, item_id, rating, timestamp.
# 


# Primeiro, remova o NumPy 2.x e instale a versão compatível
# !pip uninstall -y numpy scikit-surprise surprise
# !pip install numpy==1.24.3
# !pip install scikit-surprise==1.1.3

# Importe as bibliotecas necessárias
# !pip install scikit-surprise==1.1.3
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import cross_validate

# Carregar o dataset movielens integrado (menor versão para demonstração)
data = Dataset.load_builtin('ml-100k')

# Definir o algoritmo de recomendação (SVD - Singular Value Decomposition)
algoritmo = SVD()

# Realizar validação cruzada com 5 folds
resultados = cross_validate(algoritmo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Mostrar médias dos resultados
print("\nResultados da Validação Cruzada:")
print(f"RMSE médio: {resultados['test_rmse'].mean():.4f}")
print(f"MAE médio: {resultados['test_mae'].mean():.4f}")

# Treinar o modelo com o dataset completo
trainset = data.build_full_trainset()
algoritmo.fit(trainset)

# Exemplo de previsão (usuário 196, item 302)
usuario_id = '196'
item_id = '302'
previsao = algoritmo.predict(usuario_id, item_id)

print("\nExemplo de Previsão:")
print(f"Usuário: {previsao.uid}, Item: {previsao.iid}")
print(f"Avaliação real: {previsao.r_ui}")
print(f"Avaliação prevista: {previsao.est:.4f}")





Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9355  0.9375  0.9339  0.9345  0.9394  0.9362  0.0020  
MAE (testset)     0.7372  0.7381  0.7378  0.7367  0.7416  0.7383  0.0017  
Fit time          0.48    0.48    0.42    0.49    0.43    0.46    0.03    
Test time         0.06    0.06    0.11    0.09    0.06    0.07    0.02    

Resultados da Validação Cruzada:
RMSE médio: 0.9362
MAE médio: 0.7383

Exemplo de Previsão:
Usuário: 196, Item: 302
Avaliação real: None
Avaliação prevista: 4.0074


In [2]:
#-----------------------------------------------------------------
# Filtragem Baseada em Conteúdo com TfidfVectorizer
# Objetivo: Recomendar com base nas características dos itens.
# https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata



import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Carregar dados (substitua pelo caminho do arquivo)
df = pd.read_csv('tmdb_5000_movies.csv')  # Baixe do Kaggle

# Pré-processamento: Combinar 'overview' (descrição) + 'genres' (gêneros)
df['descricao_completa'] = df['overview'] + ' ' + df['genres'].apply(
    lambda x: ' '.join([g['name'] for g in eval(x)]) if pd.notna(x) else ''
)

# Preencher descrições vazias
df['descricao_completa'] = df['descricao_completa'].fillna('')

# Vetorização TF-IDF
vetor = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vetor.fit_transform(df['descricao_completa'])

# Matriz de similaridade
similaridade = cosine_similarity(tfidf_matrix)

# Função para recomendar filmes
def recomendar_filme(titulo, n=5):
    idx = df[df['original_title'] == titulo].index[0]
    sim_scores = list(enumerate(similaridade[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, _ in sim_scores[1:n+1]]  # Exclui o próprio filme
    
    print(f"\nRecomendações para '{titulo}':")
    for i in top_indices:
        print(f"- {df.iloc[i]['original_title']} (similaridade: {similaridade[idx][i]:.2f})")
        print(f"  Gêneros: {df.iloc[i]['genres']}")
        print(f"  Descrição: {df.iloc[i]['overview'][:100]}...\n")
        
def recomendar_filme_descricao(titulo, n=5):
    idx = df[df['overview'].str.contains(titulo, case=False, na=False)].index[0]
    sim_scores = list(enumerate(similaridade[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, _ in sim_scores[1:n+1]]  # Exclui o próprio filme
    
    print(f"\nRecomendações para '{titulo}':")
    for i in top_indices:
        print(f"- {df.iloc[i]['original_title']} (similaridade: {similaridade[idx][i]:.2f})")
        print(f"  Gêneros: {df.iloc[i]['genres']}")
        print(f"  Descrição: {df.iloc[i]['overview'][:500]}...\n")

# Testar com um filme real (ex: 'The Dark Knight')

#recomendar_filme('Avatar', n=3)

recomendar_filme_descricao('King Arthur', n=5)









Recomendações para 'King Arthur':
- Quest for Camelot (similaridade: 0.31)
  Gêneros: [{"id": 14, "name": "Fantasy"}, {"id": 16, "name": "Animation"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}, {"id": 10751, "name": "Family"}]
  Descrição: During the times of King Arthur, Kayley is a brave girl who dreams of following her late father as a Knight of the Round Table. The evil Ruber wants to invade Camelot and take the throne of King Arthur, and Kayley has to stop him....

- Arthur et les Minimoys (similaridade: 0.25)
  Gêneros: [{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 16, "name": "Animation"}, {"id": 10751, "name": "Family"}]
  Descrição: Arthur is a spirited ten-year old whose parents are away looking for work, whose eccentric grandfather has been missing for several years, and who lives with his grandmother in a country house that, in two days, will be repossessed, torn down, and turned into a block of flats unless Arthur's grandfathe

In [ ]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from sklearn.preprocessing import LabelEncoder

# --- PASSO 1: Carregar e pré-processar ---
# Certifique-se de que o arquivo está na mesma pasta do script
df = pd.read_csv('ratings_Beauty.csv')
df = df.head(10000)  # Mantendo 10k para estabilidade

# Codificação
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['UserId'])
df['item_id'] = item_encoder.fit_transform(df['ProductId'])

# --- PASSO 2: Features dos Itens (Ajustado) ---
categorias_lista = ['Skincare', 'Maquiagem', 'Cabelo', 'Perfumaria']
# Criamos um mapeamento ÚNICO: 1 produto = 1 categoria
unique_items_df = df[['item_id']].drop_duplicates()
unique_items_df['categoria'] = np.random.choice(categorias_lista, size=len(unique_items_df))

# Mesclamos de volta ao dataframe principal para manter consistência
df = df.drop(columns=['categoria'], errors='ignore').merge(unique_items_df, on='item_id')

# --- PASSO 3: Preparar o Dataset ---
dataset = Dataset()
dataset.fit(
    users=df['user_id'].unique(),
    items=df['item_id'].unique(),
    item_features=categorias_lista 
)

# Construir matriz de interações
interactions, _ = dataset.build_interactions(zip(df['user_id'], df['item_id']))

# Construir features dos itens de forma eficiente
# O formato esperado é (item_id, [lista_de_features])
item_features_tuples = [
    (row.item_id, [row.categoria])
    for row in unique_items_df.itertuples()
]
item_features = dataset.build_item_features(item_features_tuples)

# --- PASSO 4: Treinar o Modelo (Seguro para Windows/pip) ---
model = LightFM(loss='warp', no_components=20)
model.fit(
    interactions,
    item_features=item_features,
    epochs=10,
    num_threads=4
)

# --- PASSO 5: Avaliar e Recomendar ---
# Adicionado num_threads=1 também na avaliação
train_precision = precision_at_k(
    model, 
    interactions, 
    item_features=item_features, 
    k=5, 
    
).mean()

print(f"Precision@5 (treino): {train_precision:.4f}")

# Recomendações
user_id_input = 10
n_items = df['item_id'].nunique()

# Predição em lote
scores = model.predict(
    user_id_input,
    np.arange(n_items),
    item_features=item_features,
    num_threads=1
)

top_items_idx = np.argsort(-scores)[:5]
top_items_codes = item_encoder.inverse_transform(top_items_idx)

print(f"\nTop 5 recomendações para o usuário {user_id_input}:")
for i, item_code in enumerate(top_items_codes):
    cat = unique_items_df[unique_items_df['item_id'] == top_items_idx[i]]['categoria'].iloc[0]
    print(f"{i+1}. ID: {item_code} | Cat: {cat} | Score: {scores[top_items_idx[i]]:.2f}")